In [2]:
import torch
import pandas as pd
import numpy as np
from TimesNET import Model

### feature engineering ### 

from tsfracdiff import FractionalDifferentiator
import sys
sys.path.append('..')
from lib.download import *
data = get_data_features('COSUSDT').dropna()
data = data.loc[:, data.nunique() != 1]

#close j+1 est la target
target_aligned = data[['close']].copy().shift(-1).iloc[:-1]
data_aligned = data.iloc[1:-1,:4]
data_aligned["target"] = target_aligned

In [3]:
aa = get_pair_tickers()

In [4]:
import torch.nn as nn
from torch import optim
class cfg:
    def __init__(self):
        self.seq_len = 8 #longueur de la sequence 24*30*6
        self.pred_len = 1 #nombre detapes a pred, essayer 24
        self.label_len = self.pred_len
        self.d_model = 16 #dimension des deep features
        self.d_ff = 16 #dimension de skernels de l(inception)
        self.enc_in = len(data_aligned.columns) #nombre de features
        self.c_out = 1 # nombre de sorties (peut potentiellement predire plusieurs variables a la suite)
        self.e_layers = 2 #nombre de timesBlock a potentiellement augmenter avec le nombre de features
        self.embed = 'timeF' #type d'embed par défaut (sorte de time2vec series temporelles)
        self.freq = 'h' #frequence des données
        self.dropout = 0.1 #dropout pr l'embedding
        self.top_k = 2 #ombre de frequences a prendre en compte
        self.num_kernels = 6 #comme dans le modele inceptionv1
        self.learning_rate = 0.001
        self.batch_size = 16
        self.features = ['close','open']
        self.target = 'target'
        self.checkpoints = 'test'
        self.patience = 3
        self.loss = 'MSE'
        self.train_epochs = 20 #12
        self.lradj = 'type2'
        self.fracdiff = 'on'
        self.minus_n_features = 150

In [5]:
entrainé avec num_kernels = 3 ....

SyntaxError: invalid syntax (3662273608.py, line 1)

In [ ]:
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
from timefeatures import time_features

#changer timeenc pr voir
class CryptoLoader(Dataset):
    def __init__(self, df, flag='train', features=['close','open'], target='target', timeenc=0, freq='h', fracdiff='on', minus_n_features = 150):
        self.df = df.reset_index() # pour mettre le timestamp dans une nouvelle column          
        self.features = features
        self.target = target
        self.timeenc = timeenc
        self.config = cfg()
        self.freq = freq
        assert flag in ['train', 'test', 'val']
        type_map = {'train': 0, 'val': 1, 'test': 2}
        self.set_type = type_map[flag]
        self.__read_data__()
        
    def __read_data__(self):
        #filtre variables
        self.scaler = StandardScaler()
        df_raw = self.df.copy()
        cols = list(df_raw.columns)
        cols.remove(self.target)
        cols.remove('timestamp')
        
        df_raw = df_raw[['timestamp'] + cols + [self.target]]
        #calcul des ind sup et inf en fonction de : train, test, val
        
        num_train = int(len(df_raw) * 0.7)
        num_test = int(len(df_raw) * 0.2)
        num_vali = len(df_raw) - num_train - num_test
        border1s = [0, num_train - self.config.seq_len, len(df_raw) - num_test - self.config.seq_len]
        border2s = [num_train, num_train + num_vali, len(df_raw)]
        border1 = border1s[self.set_type]
        border2 = border2s[self.set_type]
        
        #pred MS => MM avec du -1
        cols_data = df_raw.columns[1:]
        df_data = df_raw[cols_data]
        
        #scale sur les données train pour eviter le feed forward bias
        train_data = df_data[border1s[0]:border2s[0]]
        self.scaler.fit(train_data.values)
        dataa = self.scaler.transform(df_data.values)
        #pas besoin de scale les features 
        
        #encodage des données temporelles
        df_stamp = df_raw[['timestamp']][border1:border2]
        df_stamp['timestamp'] = pd.to_datetime(df_stamp.timestamp)
        if self.timeenc == 0:
            df_stamp['month'] = df_stamp.timestamp.apply(lambda row: row.month, 1)
            df_stamp['day'] = df_stamp.timestamp.apply(lambda row: row.day, 1)
            df_stamp['weekday'] = df_stamp.timestamp.apply(lambda row: row.weekday(), 1)
            df_stamp['hour'] = df_stamp.timestamp.apply(lambda row: row.hour, 1)
            data_stamp = df_stamp.drop(['timestamp'], 1).values
        elif self.timeenc == 1:
            data_stamp = time_features(pd.to_datetime(df_stamp['timestamp'].values), freq=self.freq)
            data_stamp = data_stamp.transpose(1, 0)
        self.data_x = dataa[border1:border2]
        self.data_y = dataa[border1:border2]
        self.data_stamp = data_stamp
        
        
    def __getitem__(self, index):
        s_begin = index
        s_end = s_begin + self.config.seq_len
        r_begin = s_end - self.config.label_len
        r_end = r_begin + self.config.label_len + self.config.pred_len
        seq_x = self.data_x[s_begin:s_end]
        seq_y = self.data_y[r_begin:r_end]
        seq_x_mark = self.data_stamp[s_begin:s_end]
        seq_y_mark = self.data_stamp[r_begin:r_end]
        return seq_x, seq_y, seq_x_mark, seq_y_mark
        
    def __len__(self):
        return len(self.data_x) - self.config.seq_len - self.config.pred_len + 1

    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)

In [ ]:
from torch.utils.data import DataLoader

def data_provider(df, args, flag):
    timeenc = 0 if args.embed != 'timeF' else 1
    shuffle_flag = True if flag != 'test' else False
    drop_last = True

    if flag == 'test':
        drop_last = False
        batch_size = 1  # bsz=1 for evaluation
    else:
        batch_size = args.batch_size  # bsz for train and valid
    
    data_set = CryptoLoader(
        df=df,
        flag=flag,
        features=args.features,
        target=args.target,
        timeenc=timeenc,
    )

    data_loader = DataLoader(
        data_set,
        batch_size=batch_size,
        shuffle=shuffle_flag,
        drop_last=drop_last
    )
    return data_set, data_loader

In [6]:
import torch as t
class smape_loss(nn.Module):
    def __init__(self):
        super(smape_loss, self).__init__()

    def forward(self, insample: t.Tensor, freq: int,
                forecast: t.Tensor, target: t.Tensor, mask: t.Tensor) -> t.float:
        return 200 * t.mean(divide_no_nan(t.abs(forecast - target),
                                          t.abs(forecast.data) + t.abs(target.data)) * mask)
def adjust_learning_rate(optimizer, epoch, args):
    # lr = args.learning_rate * (0.2 ** (epoch // 2))
    if args.lradj == 'type1':
        lr_adjust = {epoch: args.learning_rate * (0.5 ** ((epoch - 1) // 1))}
    elif args.lradj == 'type2':
        lr_adjust = {
            15: 1e-5,
        }
    if epoch in lr_adjust.keys():
        lr = lr_adjust[epoch]
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        print('Updating learning rate to {}'.format(lr))
        
def divide_no_nan(a, b):
    result = a / b
    result[result != result] = .0
    result[result == np.inf] = .0
    return result
        
def visual(true, preds=None, name='./pic/test.pdf'):
    plt.figure()
    plt.plot(true, label='GroundTruth', linewidth=2)
    if preds is not None:
        plt.plot(preds, label='Prediction', linewidth=2)
    plt.legend()
    plt.savefig(name, bbox_inches='tight')
        
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model, path):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model, path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model, path)
            self.counter = 0

    def save_checkpoint(self, val_loss, model, path):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), path + '/' + 'checkpoint.pth')
        self.val_loss_min = val_loss


In [7]:
class Crypto_Pred_Exp:
    def __init__(self, df, args):
        self.args = cfg()
        self.df = df
        self.device = torch.device('cuda')
        self.frequency_map = 24
        self.model = self._build_model()
        
    def _build_model(self):
        model = Model(self.args).to(self.device)  
        return model.float()
    
    def _get_data(self, flag):
        data_set, data_loader = data_provider(self.df,self.args, flag)
        return data_set, data_loader
    
    def _select_optimizer(self):
        model_optim = optim.Adam(self.model.parameters(), lr=self.args.learning_rate)
        return model_optim
    
    def _select_criterion(self, loss_name):
        if loss_name == 'MSE':
            return nn.MSELoss()
        elif loss_name == 'SMAPE':
            return smape_loss()
        
    def vali(self, vali_data, vali_loader, criterion):
        total_loss = []
        self.model.eval()
        with torch.no_grad():
            for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(vali_loader):
                batch_x = batch_x.float().to(self.device)
                batch_y = batch_y.float()

                batch_x_mark = batch_x_mark.float().to(self.device)
                batch_y_mark = batch_y_mark.float().to(self.device)

                # decoder input
                dec_inp = torch.zeros_like(batch_y[:, -self.args.pred_len:, :]).float()
                dec_inp = torch.cat([batch_y[:, :self.args.label_len, :], dec_inp], dim=1).float().to(self.device)
                
                # encoder - decoder
                outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

                f_dim = -1
                outputs = outputs[:, -self.args.pred_len:, f_dim:]
                batch_y = batch_y[:, -self.args.pred_len:, f_dim:].to(self.device)

                pred = outputs.detach().cpu()
                true = batch_y.detach().cpu()

                loss = criterion(pred, true)

                total_loss.append(loss)
        total_loss = np.average(total_loss)
        self.model.train()
        return total_loss

    def train(self, setting):
        train_data, train_loader = self._get_data(flag='train')
        vali_data, vali_loader = self._get_data(flag='val')
        test_data, test_loader = self._get_data(flag='test')

        path = os.path.join(self.args.checkpoints, setting)
        if not os.path.exists(path):
            os.makedirs(path)

        time_now = time.time()

        train_steps = len(train_loader)
        early_stopping = EarlyStopping(patience=self.args.patience, verbose=True)

        model_optim = self._select_optimizer()
        criterion = self._select_criterion(self.args.loss)

        for epoch in range(self.args.train_epochs):
            iter_count = 0
            train_loss = []

            self.model.train()
            epoch_time = time.time()
            for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(train_loader):
                iter_count += 1
                model_optim.zero_grad()
                batch_x = batch_x.float().to(self.device)

                batch_y = batch_y.float().to(self.device)
                batch_x_mark = batch_x_mark.float().to(self.device)
                batch_y_mark = batch_y_mark.float().to(self.device)

                # decoder input
                dec_inp = torch.zeros_like(batch_y[:, -self.args.pred_len:, :]).float()
                dec_inp = torch.cat([batch_y[:, :self.args.label_len, :], dec_inp], dim=1).float().to(self.device)

                # encoder - decoder
                outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
                f_dim = -1
                outputs = outputs[:, -self.args.pred_len:, f_dim:]
                batch_y = batch_y[:, -self.args.pred_len:, f_dim:].to(self.device)
                loss = criterion(outputs, batch_y)
                train_loss.append(loss.item())

                if (i + 1) % 100 == 0:
                    print("\titers: {0}, epoch: {1} | loss: {2:.7f}".format(i + 1, epoch + 1, loss.item()))
                    speed = (time.time() - time_now) / iter_count
                    left_time = speed * ((self.args.train_epochs - epoch) * train_steps - i)
                    print('\tspeed: {:.4f}s/iter; left time: {:.4f}s'.format(speed, left_time))
                    iter_count = 0
                    time_now = time.time()
                    loss.backward()
                    model_optim.step()

            print("Epoch: {} cost time: {}".format(epoch + 1, time.time() - epoch_time))
            train_loss = np.average(train_loss)
            vali_loss = self.vali(vali_data, vali_loader, criterion)
            test_loss = self.vali(test_data, test_loader, criterion)

            print("Epoch: {0}, Steps: {1} | Train Loss: {2:.7f} Vali Loss: {3:.7f} Test Loss: {4:.7f}".format(
                epoch + 1, train_steps, train_loss, vali_loss, test_loss))
            early_stopping(vali_loss, self.model, path)
            if early_stopping.early_stop:
                print("Early stopping")
                break

            adjust_learning_rate(model_optim, epoch + 1, self.args)

        best_model_path = path + '/' + 'checkpoint.pth'
        self.model.load_state_dict(torch.load(best_model_path))

        return self.model

    def test(self, setting, test=0):
        test_data, test_loader = self._get_data(flag='test')
        if test:
            print('loading model')
            self.model.load_state_dict(torch.load(os.path.join('./checkpoints/' + setting, 'checkpoint.pth')))

        preds = []
        trues = []
        folder_path = './test_results/' + setting + '/'
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        self.model.eval()
        with torch.no_grad():
            for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(test_loader):
                batch_x = batch_x.float().to(self.device)
                batch_y = batch_y.float().to(self.device)

                batch_x_mark = batch_x_mark.float().to(self.device)
                batch_y_mark = batch_y_mark.float().to(self.device)

                # decoder input
                dec_inp = torch.zeros_like(batch_y[:, -self.args.pred_len:, :]).float()
                dec_inp = torch.cat([batch_y[:, :self.args.label_len, :], dec_inp], dim=1).float().to(self.device)
                # encoder - decoder
                outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

                f_dim = -1
                outputs = outputs[:, -self.args.pred_len:, f_dim:]
                batch_y = batch_y[:, -self.args.pred_len:, f_dim:].to(self.device)
                outputs = outputs.detach().cpu().numpy()
                batch_y = batch_y.detach().cpu().numpy()

                pred = outputs
                true = batch_y

                preds.append(pred)
                trues.append(true)
                if i % 20 == 0:
                    input = batch_x.detach().cpu().numpy()
                    gt = np.concatenate((input[0, -100:, -1], true[0, -100:, -1]), axis=0)
                    pd = np.concatenate((input[0, -100:, -1], pred[0, -100:, -1]), axis=0)
                    visual(gt, pd, os.path.join(folder_path, str(i) + '.pdf'))

        preds = np.array(preds)
        trues = np.array(trues)
        print('test shape:', preds.shape, trues.shape)
        preds = preds.reshape(-1, preds.shape[-2], preds.shape[-1])
        trues = trues.reshape(-1, trues.shape[-2], trues.shape[-1])
        print('test shape:', preds.shape, trues.shape)

        # result save
        folder_path = './results/' + setting + '/'
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        mae, mse, rmse, mape, mspe = metric(preds, trues)
        print('mse:{}, mae:{}'.format(mse, mae))
        f = open("result_long_term_forecast.txt", 'a')
        f.write(setting + "  \n")
        f.write('mse:{}, mae:{}'.format(mse, mae))
        f.write('\n')
        f.write('\n')
        f.close()

        np.save(folder_path + 'metrics.npy', np.array([mae, mse, rmse, mape, mspe]))
        np.save(folder_path + 'pred.npy', preds)
        np.save(folder_path + 'true.npy', trues)

        return

In [8]:
import matplotlib.pyplot as plt

def RSE(pred, true):
    return np.sqrt(np.sum((true - pred) ** 2)) / np.sqrt(np.sum((true - true.mean()) ** 2))


def CORR(pred, true):
    u = ((true - true.mean(0)) * (pred - pred.mean(0))).sum(0)
    d = np.sqrt(((true - true.mean(0)) ** 2 * (pred - pred.mean(0)) ** 2).sum(0))
    return (u / d).mean(-1)


def MAE(pred, true):
    return np.mean(np.abs(pred - true))


def MSE(pred, true):
    return np.mean((pred - true) ** 2)


def RMSE(pred, true):
    return np.sqrt(MSE(pred, true))


def MAPE(pred, true):
    return np.mean(np.abs((pred - true) / true))


def MSPE(pred, true):
    return np.mean(np.square((pred - true) / true))


def metric(pred, true):
    mae = MAE(pred, true)
    mse = MSE(pred, true)
    rmse = RMSE(pred, true)
    mape = MAPE(pred, true)
    mspe = MSPE(pred, true)
    return mae, mse, rmse, mape, mspe

In [10]:
config = cfg()

In [11]:
for ticker in tqdm(aa[194:]):
    try:
        data = get_data_features(ticker).dropna()
        data = data.loc[:, data.nunique() != 1]

        #close j+1 est la target
        target_aligned = data[['close']].copy().shift(-1).iloc[:-1]
        data_aligned = data.iloc[1:-1,:4]
        data_aligned["target"] = target_aligned
        exp = Crypto_Pred_Exp(data_aligned, config)
        exp.train(f'{ticker}_ohlc_8_top2_samelr')
        exp.test(f'{ticker}_ohlc_8_top2_samelr')
    except:
        pass

  1%|          | 3/256 [00:08<15:33,  3.69s/it]

Crypto Morte Date dépassée LTCUPUSDT derniere date : 2022-01-27 02:00:00 taille : (11925, 11)
Probleme Indicator
ERROR LTCUPUSDT ouverture fichier feature


  2%|▏         | 4/256 [00:15<21:26,  5.11s/it]

Crypto Morte Date dépassée LTCDOWNUSDT derniere date : 2022-01-27 02:00:00 taille : (11901, 11)
Probleme Indicator
ERROR LTCDOWNUSDT ouverture fichier feature


  3%|▎         | 7/256 [00:27<17:23,  4.19s/it]

...

Crypto Morte Date dépassée COMBOUSDT derniere date : 2023-07-03 16:00:00 taille : (753, 11)
Probleme Indicator
ERROR COMBOUSDT ouverture fichier feature


100%|█████████▉| 255/256 [06:25<00:00,  1.73it/s]

Crypto Morte Date dépassée MAVUSDT derniere date : 2023-07-03 16:00:00 taille : (129, 11)
Probleme Indicator
ERROR MAVUSDT ouverture fichier feature


100%|██████████| 256/256 [06:25<00:00,  1.51s/it]

Crypto Morte Date dépassée PENDLEUSDT derniere date : 2023-07-03 16:00:00 taille : (7, 11)
Probleme Indicator
ERROR PENDLEUSDT ouverture fichier feature


In [ ]:
recup les saisons

cluster features tres polyvalent - roulant   caracteristiques - tendance moyenne du cluster - correlation intercluster - correlation marché global

In [ ]:
faire un enorme dataframe avec ttes cryptos a feed au timesnet qui classifie en sortie une coposition de portefeuille et comme reward le sharpe ratio

In [ ]:
Informer
https://huggingface.co/docs/transformers/main/model_doc/informer
    
tester pleins de modeles

seq[0] contient X lignes avec l'ensemble des features avec X=seq_len
=> chaque indice de l'objet sequence contient les features des X derniers jours

In [ ]:
peut lui feed plusieurs monnaies et lui demander de predire la composition parfaite par ex

In [ ]:
https://github.com/kyegomez/Sophia
    
    
tester Informer pr les dependences entre les variables


car timesnet fait intervenir les relations entre les données indirectement en utillisant une some alors que informer peut directement capturer des features dans le temps entres ces variables

In [2]:
fracDiff = FractionalDifferentiator()
frac_data = fracDiff.FitTransform(data).dropna()
frac_data_target = frac_data[['close']].copy().shift(-1).iloc[:-1]
frac_data_aligned = frac_data.iloc[:-1,:4]#-150]
frac_data_aligned["target"] = frac_data_target